In [ ]:
# /bin/sh
# ----------------Parameters---------------------- #
#$ -S /bin/sh
#$ -pe mthread 15
#$ -q sThC.q
#$ -l mres=120G,h_data=8G,h_vmem=8G
#$ -cwd
#$ -j y
#$ -N index_oyster
#$ -o index_oyster.log
#$ -M gignoux-wolfsohns@si.edu
#
# ----------------Modules------------------------- #
module load bioinformatics/bowtie2
#
# ----------------Your Commands------------------- #
#
echo + `date` job $JOB_NAME started in $QUEUE with jobID=$JOB_ID on $HOSTNAME
echo + NSLOTS = $NSLOTS
#
bowtie2-build GCA_002022765.4_C_virginica-3.0_genomic.fna C_virginica-3.0_genomic.fna_index --threads 15
#
echo = `date` job $JOB_NAME done

In [ ]:
| sed 's/[^-]*$//'

In [ ]:
# /bin/sh 
# ----------------Parameters---------------------- #
#$  -S /bin/sh
#$ -pe mthread 12
#$ -q mThC.q
#$ -l mres=72G,h_data=6G,h_vmem=6G
#$ -cwd
#$ -j y
#$ -N trim_ce
#$ -o trim_ce.log
#$ -m bea
#$ -M gignoux-wolfsohns@si.edu
#
# ----------------Modules------------------------- #
module load bioinformatics/trim_galore
#
# ----------------Your Commands------------------- #
#
echo + `date` job $JOB_NAME started in $QUEUE with jobID=$JOB_ID on $HOSTNAME
echo + NSLOTS = $NSLOTS
#
for f in $(ls | sed 's/[^-]*$//'); 
do 
    trim_galore -q 20 --phred33 --fastqc --illumina --length 20 --paired "$f"CV_R1_001.fastq "$f"CV_R2_001.fastq;
done

#
echo = `date` job $JOB_NAME done

Ok, so groves had these degenerate bases but we dont have that, and our adapters were already removed since they are just the standard illumina adapters. So I think just go straight to mapping adn then use picard tools to remove duplicates on bam files? 

In [ ]:
# /bin/sh
# ----------------Parameters---------------------- #
#$ -S /bin/sh
#$ -pe mthread 12
#$ -q mThC.q
#$ -l mres=72G,h_data=6G,h_vmem=6G
#$ -cwd
#$ -j y
#$ -N map_CE
#$ -o map_CE.log
#$ -m bea
#$ -M gignoux-wolfsohns@si.edu
#
# ----------------Modules------------------------- #
module load bioinformatics/bowtie2
#
# ----------------Your Commands------------------- #
#
echo + `date` job $JOB_NAME started in $QUEUE with jobID=$JOB_ID on $HOSTNAME
echo + NSLOTS = $NSLOTS
#


for f in $(ls | sed 's/[^-]*$//'); 
do 

bowtie2 -x /pool/genomics/gignoux-wolfsohns/Reference_genomes/cvirginica_genome/C_virginica-3.0_genomic.fna_index -1 /pool/genomics/gignoux-wolfsohns/CE_MR/trimmed_seqs/"$f"CV_R1_001_val_1.fq -2 /pool/genomics/gignoux-wolfsohns/CE_MR/trimmed_seqs/"$f"CV_R2_001_val_2.fq -p 12 -S /pool/genomics/gignoux-wolfsohns/CE_MR/mapped_sam/"$f".sam; 

done

#
echo = `date` job $JOB_NAME done


Trying with --very-sensitive --local, should give higher mapping percentages. 
+ Fri Dec 17 17:46:49 EST 2021 job map_CE_vsl started in mThC.q with jobID=1727340 on compute-43-22
+ NSLOTS = 12
90.96% overall alignment rate
90.96% overall alignment rate
89.81% overall alignment rate
89.81% overall alignment rate
90.43% overall alignment rate
90.43% overall alignment rate
90.38% overall alignment rate
90.38% overall alignment rate
90.05% overall alignment rate
90.05% overall alignment rate
89.81% overall alignment rate
89.81% overall alignment rate
90.56% overall alignment rate
90.56% overall alignment rate
90.42% overall alignment rate
90.42% overall alignment rate
91.91% overall alignment rate
91.91% overall alignment rate
85.87% overall alignment rate
85.87% overall alignment rate
88.27% overall alignment rate
88.27% overall alignment rate
90.20% overall alignment rate
90.20% overall alignment rate
91.47% overall alignment rate
91.47% overall alignment rate
90.89% overall alignment rate
90.89% overall alignment rate
90.78% overall alignment rate
90.78% overall alignment rate
89.78% overall alignment rate
89.78% overall alignment rate
86.83% overall alignment rate
86.83% overall alignment rate
89.04% overall alignment rate
89.04% overall alignment rate
89.27% overall alignment rate
89.27% overall alignment rate
75.95% overall alignment rate
75.95% overall alignment rate
87.45% overall alignment rate
87.45% overall alignment rate
89.95% overall alignment rate
89.95% overall alignment rate
85.63% overall alignment rate
85.63% overall alignment rate
86.13% overall alignment rate
86.13% overall alignment rate
89.49% overall alignment rate
89.49% overall alignment rate
87.43% overall alignment rate
87.43% overall alignment rate
89.07% overall alignment rate
89.07% overall alignment rate
88.73% overall alignment rate
88.73% overall alignment rate
88.70% overall alignment rate
88.70% overall alignment rate
81.82% overall alignment rate
81.82% overall alignment rate

In [ ]:
Next run picard tools mark duplicates

In [ ]:
# /bin/sh
# ----------------Parameters---------------------- #
#$ -S /bin/sh
#$ -q mThC.q
#$ -l mres=8G,h_data=8G,h_vmem=8G
#$ -cwd
#$ -j y
#$ -N sam2bamCE
#$ -o sam2bamCE.log
#
# ----------------Modules------------------------- #
module load bioinformatics/samtools
#
# ----------------Your Commands------------------- #
#
echo + `date` job $JOB_NAME started in $QUEUE with jobID=$JOB_ID on $HOSTNAME
#

for f in $(ls | sed 's/[^-]*$//'); 
do 
samtools view -b "$f".sam > /pool/genomics/gignoux-wolfsohns/CE_MR/mapped_bam/"$f".bam;
done
#
echo = `date` job $JOB_NAME done

In [ ]:
for f in $(ls | sed 's/[^-]*$//'); 
do 

java -jar picard.jar MarkDuplicates \
      I="$f".bam \
      O="$f"marked_duplicates.bam \
      M="$f"marked_dup_metrics.txt;
done


In [ ]:
# /bin/sh
# ----------------Parameters---------------------- #
#$ -S /bin/sh
#$ -q sThC.q
#$ -l mres=8G,h_data=8G,h_vmem=8G
#$ -cwd
#$ -j y
#$ -N Dup_CE
#$ -o Dup_CE.log
#
# ----------------Modules------------------------- #
module load bioinformatics/picard-tools
#
# ----------------Your Commands------------------- #
#
echo + `date` job $JOB_NAME started in $QUEUE with jobID=$JOB_ID on $HOSTNAME
#
for f in $(ls | sed 's/[^-]*$//'); 
do 
java -jar picard.jar MarkDuplicates \
      I="$f".bam \
      O="$f"marked_duplicates.bam \
      M="$f"marked_dup_metrics.txt;
done
#
echo = `date` job $JOB_NAME done